In [2]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import matplotlib.pyplot as plt
import os
from astropy.time import Time

/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/gwcosmo/utilities/cosmology.py:20: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [3]:
def fakeGeoCentTime():
    """
    A function to generate a fake geocentric time between 2020-2022 (inclusive)
    returns the geocentric time in unix form
    """
    times = ["2020-01-01T00:00:00.123456789", "2023-01-01T00:00:00"]
    tArr = Time(times, format="isot", scale="tcg")
    return bb.core.prior.Uniform(tArr[0], tArr[1]).sample(1).unix[0]


def thresholdChecker(interferometers, injection, thresholds=(8, 2)):
    network_snr, indiv_snr = 0, []
    for ifo in interferometers:
        signal = ifo.get_detector_response(
            waveform_polarizations=waveform_generator.frequency_domain_strain(
                injection
            ),
            parameters=injection,
        )
        single_snr_squared = ifo.optimal_snr_squared(signal).real
        network_snr += single_snr_squared
        indiv_snr.append(single_snr_squared)
    network_snr = np.sqrt(network_snr)
    if network_snr > thresholds[0] and all([x > thresholds[1] for x in indiv_snr]):
        return True
    else:
        return False

In [4]:
# Getting the priors
prior = bb.gw.prior.BBHPriorDict(aligned_spin=False)

11:53 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [5]:
# Turning entries in the prior dictionary from Constraint type to Uniform over the constraint range
for i, k in zip(prior.keys(), prior.values()):
    if type(k) == bb.core.prior.base.Constraint:
        prior[i] = bb.core.prior.Uniform(k.minimum, k.maximum)

In [6]:
# Populating the waveform dictionary
waveform_arguments = dict(
    waveform_approximant="IMRPhenomXP",  # waveform approximant name
    reference_frequency=50.0,  # gravitational waveform reference frequency (Hz)
)

# Setting some parameters for the waveform generator
duration = 4
sampling_frequency = 2048

# Calling all three interferometers for these purposes
ifos = bb.gw.detector.InterferometerList(["H1", "V1", "L1"])

# # Injecting a signal into the three interferometers
# _ = ifos.inject_signal(
#     waveform_generator=waveform_generator, parameters=injections)

In [11]:
for k in range(100):
    inj = prior.sample()

    # # Adding a geocentTime
    inj["geocent_time"] = fakeGeoCentTime()

    # Creating the waveform generator
    waveform_generator = bb.gw.waveform_generator.WaveformGenerator(
        sampling_frequency=sampling_frequency,
        duration=duration,
        frequency_domain_source_model=bb.gw.source.lal_binary_black_hole,
        parameters=inj,
        waveform_arguments=waveform_arguments,
    )

    # Setting strain on the interferometers
    ifos.set_strain_data_from_power_spectral_densities(
        duration=duration,
        sampling_frequency=sampling_frequency,
        start_time=inj["geocent_time"] - 2,
    )
    while thresholdChecker(ifos, inj):
        inj = prior.sample()
        # # Adding a geocentTime
        inj["geocent_time"] = fakeGeoCentTime()

        # Creating the waveform generator
        waveform_generator = bb.gw.waveform_generator.WaveformGenerator(
            sampling_frequency=sampling_frequency,
            duration=duration,
            frequency_domain_source_model=bb.gw.source.lal_binary_black_hole,
            parameters=inj,
            waveform_arguments=waveform_arguments,
        )

        # Setting strain on the interferometers
        ifos.set_strain_data_from_power_spectral_densities(
            duration=duration,
            sampling_frequency=sampling_frequency,
            start_time=inj["geocent_time"] - 2,
        )
        network_snr = 0
        for ifo in ifos:
            signal = ifo.get_detector_response(
                waveform_polarizations=waveform_generator.frequency_domain_strain(inj),
                parameters=inj,
            )
            single_snr_squared = ifo.optimal_snr_squared(signal).real
            network_snr += single_snr_squared
        network_snr = np.sqrt(network_snr)

11:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
11:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
11:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
11:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
11:54 bi

In [12]:
network_snr

0.8371482412347266

In [13]:
single_snr_squared

0.4617288154662496

In [88]:
ifos[0].matched_filter_snr(sig["plus"]), ifos[0].matched_filter_snr(sig["cross"])

((-0.7629343788194614-0.8896101127313927j),
 (0.8431112172454988-0.6975967946560718j))

In [89]:
ifos[0].matched_filter_snr(ifos[0].whitened_time_domain_strain)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 8192 but corresponding boolean dimension is 4097

In [ ]:
np.shape(ifos[0].whitened_time_domain_strain)